In [7]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
LOCAL_FOLDER = "G:/notebook/MNIST_data/"
data = input_data.read_data_sets(LOCAL_FOLDER, one_hot=True, validation_size=0)

Extracting G:/notebook/MNIST_data/train-images-idx3-ubyte.gz
Extracting G:/notebook/MNIST_data/train-labels-idx1-ubyte.gz
Extracting G:/notebook/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting G:/notebook/MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
np.random.seed(10)
tf.set_random_seed(10)

class MLP(object):
    def __init__(self, input, in_dim, out_dim, act=tf.nn.relu, vn=('W', 'b')):
        self.input = input
        
        W = tf.Variable(
            tf.random_uniform(
                [in_dim, out_dim], 
                maxval = tf.sqrt(6.0) / tf.sqrt(float(out_dim+in_dim)),
                seed = 10
            ), name=vn[0]
        )
        
        b = tf.Variable(tf.zeros([out_dim]), name=vn[1])
        
        self.W = W
        self.b = b
        self.params = [self.W, self.b]
        self.act = act
    
    def output(self):
        y = tf.matmul(self.input, self.W) + self.b
        self.output = self.act(y)
        
        return self.output

In [13]:
input = tf.placeholder(tf.float32, [None, 784], name="input")
targets = tf.placeholder(tf.float32, [None, 10], name="targets")

hidden1 = MLP(input, 784, 200, act=tf.nn.relu, vn=("W1", "b1"))

output = MLP(hidden1.output(), 200, 10, act=tf.identity, vn=("W2", "b2"))

predict = output.output()

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=targets, logits=predict
    )
)

#loss = cross_entropy + 1e-6 * (tf.nn.l2_loss(hidden1.W) + tf.nn.l2_loss(output.W))
#loss = cross_entropy + 1e-6 * (tf.nn.l1_loss(hidden1.W) + tf.nn.l1_loss(output.W))
loss = cross_entropy + 1e-6 * (tf.reduce_sum(tf.abs(hidden1.W)) + tf.reduce_sum(tf.abs(output.W)))
#loss = cross_entropy + 1e-6 * (tf.reduce_sum(tf.abs(hidden1.W)) + tf.reduce_sum(tf.abs(output.W)))+ 1e-6 * (tf.nn.l2_loss(hidden1.W) + tf.nn.l2_loss(output.W))

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [14]:
from matplotlib import pyplot

steps = []
accuracies = []

# creating session
sess = tf.InteractiveSession()

# initializing trainable variables
sess.run(tf.global_variables_initializer())

# training loop
for step in range(4000):
    # fetching next batch of training data
    batch_xs, batch_ys = data.train.next_batch(100)

    if step % 100 == 0:
        # reporting current accuracy of the model on every 100th batch
        batch_accuracy = sess.run(accuracy, feed_dict={input: batch_xs, targets: batch_ys})
        #print("{0}:\tbatch accuracy {1:.2f}".format(step, batch_accuracy))
        steps.append(step/100)
        accuracies.append(batch_accuracy)

    # running the training step with the fetched batch
    sess.run(train_step, feed_dict={input: batch_xs, targets: batch_ys})

pyplot.plot(steps, accuracies, 'k-', linewidth=2.0, color='green')
pyplot.show()   

In [15]:
# evaluating model prediction accuracy of the model on the test set
test_accuracy = sess.run(accuracy, feed_dict={input: data.test.images, targets: data.test.labels})


print("-------------------------------------------------")
print("Test set accuracy: {0:.4f}".format(test_accuracy))

-------------------------------------------------
Test set accuracy: 0.9574
